In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

from numpy import logical_and as npand
from numpy import logical_or as npor
import matplotlib

In [ ]:
from sympy import (symbols, pi, I, E, cos, sin, exp, tan, simplify, expand, factor, collect,
                   apart, cancel, expand_trig, diff, Derivative, Function, integrate, limit,
                   series, Eq, solve, dsolve, Matrix, N, preorder_traversal, Float, solve_linear_system,
                   eye, zeros, lambdify, Symbol,hessian, sqrt)
from sympy.physics.mechanics import dynamicsymbols, init_vprinting

In [ ]:
init_vprinting()

In [ ]:
from sympy.physics.mechanics import ReferenceFrame, Point, LagrangesMethod, Lagrangian, inertia, RigidBody, dot

In [ ]:
N = ReferenceFrame('N')
origen = Point('origen')
origen.set_vel(N, 0)


In [ ]:
t = symbols('t')
t

In [ ]:
x, y, psi = dynamicsymbols('x y psi')
x, y, psi

In [ ]:
fi0, fi1, fi2, fi3 = dynamicsymbols('phi:4')
fi0, fi1, fi2, fi3

In [ ]:
CM = origen.locatenew('CM', x*N.x + y*N.y)
CM.set_vel(N, CM.pos_from(origen).dt(N))

In [ ]:
B = N.orientnew('B', 'Axis', (psi, N.z))

In [ ]:
m_r, m_w, i_r, i_w, l0, l1, r = symbols('m_r m_w i_r i_w l0 l1 r')

In [ ]:
I_r = inertia(B, 0, 0, i_r)
body = RigidBody('Body', CM, B, m_r, (I_r,CM))

In [ ]:
wheel_params = [
    [45,  l0,  l1, fi0],
    [-45, l0, -l1, fi1],
    [-45, -l0, l1, fi2],
    [45, -l0, -l1, fi3]
]

In [ ]:
wheel_axes = []
wheel_centers = []
I_w = inertia(B, 0, i_w, 0)
wheel_bodies = []
for ii in range(4):
    wheel_axis = B.orientnew(f'W{ii}', 'Axis', (wheel_params[ii][3], B.y))
    wheel_axes.append(wheel_axis)
    wheel_center = CM.locatenew(f'CMW{ii}', wheel_params[ii][1]*B.x + wheel_params[ii][2]*B.y)
    wheel_center.set_vel(N, wheel_center.v2pt_theory(CM, N, B))
    wheel_centers.append(wheel_center)
    wheel_body = RigidBody(f'Wheel{ii}', wheel_center, wheel_axis, 0, (I_w,wheel_center))
    wheel_bodies.append(wheel_body)
wheel_axes, wheel_centers, wheel_bodies

In [ ]:
lag_f = simplify(Lagrangian(N, body, *wheel_bodies))
lag_f

## Restrictions:

In [ ]:
from optibot.symbolic import integerize

In [ ]:
non_hol_rest = []
for ii in range(4):
    angle = np.radians(wheel_params[ii][0])
    aux_syst = B.orientnew('aux', 'Axis', (angle, B.z))
    contact_p = wheel_centers[ii].locatenew('contact_p', -r*N.z)
    contact_p.set_vel(N, contact_p.v2pt_theory(wheel_centers[ii], N, wheel_axes[ii]))
    raw_restr = dot(contact_p.vel(N),aux_syst.y)
    non_hol_rest.append(simplify(integerize((2/2**0.5)* raw_restr)))

In [ ]:
Matrix(non_hol_rest)

## Lagrange system

In [ ]:
FL = []
for ii in range(4):
    u = symbols(f'u{ii}')
    FL.append((wheel_axes[ii], u*B.y))

In [ ]:
LM = LagrangesMethod(lag_f, [x, y, psi, fi0, fi1, fi2, fi3] , forcelist=FL, frame=N, nonhol_coneqs=non_hol_rest)

In [ ]:
LM.form_lagranges_equations()

In [ ]:
M = LM.mass_matrix[:,:7]
M_in = M[:3,:3]
M_de = M[3:,3:]
M, M_in, M_de

In [ ]:
phi_q = LM.lam_coeffs
phi_q_in = phi_q[:,:3]
phi_q_de = phi_q[:,3:]
phi_q, phi_q_in, phi_q_de

In [ ]:
Q = LM.forcing
Q_in = Matrix(Q[:3])
Q_de = Matrix(Q[3:])
Q, Q_in, Q_de

In [ ]:
q_dot = LM.q.diff(t)
q_dot

In [ ]:
q_dotdot = q_dot.diff(t)
q_dotdot

In [ ]:
q_dot_in = Matrix(q_dot[:3])
q_dotdot_in = Matrix(q_dotdot[:3])
q_dot_in, q_dotdot_in

In [ ]:
phi_q_de_inv = simplify(phi_q_de.pinv())
phi_q_de_inv

In [ ]:
R = simplify(- phi_q_de_inv @ phi_q_in)
R

In [ ]:
R_dot = R.diff(t)
R_dot

In [ ]:
phi_q_de_inv@phi_q_de.diff(t)@R+phi_q_de_inv@phi_q_in.diff(t)

In [ ]:
q_dot_de = R@q_dot_in
q_dot_de

In [ ]:
H = simplify(M_in + R.T @ M_de @ R)
H

In [ ]:
K = simplify(R.T @ M_de @ R_dot)
K

In [ ]:
Fa = simplify(Q_in + R.T@Q_de)
Fa

In [ ]:
q_dotdot_in_expr = simplify(H.pinv() @ (Fa - K@q_dot_in))
q_dotdot_in_expr

In [ ]:
q_dotdot_de_expr = simplify(R_dot @ q_dot_in + R @ q_dotdot_in_expr)
q_dotdot_de_expr

In [ ]:
lambda_vec = simplify(phi_q_de_inv@(Q_de- M_de@q_dotdot_de_expr))
lambda_vec

In [ ]:
len(LM.coneqs)

In [ ]:
class SimpLagrangesMethod():
    def __init__(self, Lagrangian, qs, forcelist=None, bodies=None, frame=None,
                 hol_coneqs=None, nonhol_coneqs=None, simplif=True, print_status=True):
        
        self.print_status = print_status
        self.LM = LagrangesMethod(Lagrangian, qs, forcelist, bodies, frame, hol_coneqs, nonhol_coneqs)
        if print_status: print('Generating Lagrange Equations')
        self.LM.form_lagranges_equations()
        
        n = len(qs)
        t = symbols('t')
        self.M = self.LM.mass_matrix[:,:n]
        self.Q = self.LM.forcing
        self.q_dot = self.LM.q.diff(t)
        self.forcelist = forcelist
        self.coneqs = self.LM.coneqs
        
        #print(self.coneqs,len(self.coneqs))
        if len(self.coneqs)> 0:
            m = len(self.coneqs)
            n_ind = n-m

            self.M = self.LM.mass_matrix[:,:n]
            self.M_in = self.M[:n_ind,:n_ind]
            self.M_de = self.M[n_ind:,n_ind:]

            self.phi_q = self.LM.lam_coeffs
            self.phi_q_in = self.phi_q[:,:n_ind]
            self.phi_q_de = self.phi_q[:,n_ind:]

            self.Q_in = Matrix(self.Q[:n_ind])
            self.Q_de = Matrix(self.Q[n_ind:])

            self.q_dot_in = Matrix(self.q_dot[:n_ind])

            if print_status: print('Generating and simplifiying Phi_q_de_inv')
            self.phi_q_de_inv = simplify(self.phi_q_de.pinv())
            if print_status: print('Generating and simplifiying R')
            self.R = simplify(- self.phi_q_de_inv @ self.phi_q_in)
            self.R_dot = self.R.diff(t)
            self.q_dot_de = self.R@self.q_dot_in
            if print_status: print('Generating and simplifiying H')
            self.H = simplify(self.M_in + self.R.T @ self.M_de @ self.R)
            if print_status: print('Generating and simplifiying K')
            self.K = simplify(self.R.T @ self.M_de @ self.R_dot)
            if print_status: print('Generating and simplifiying Fa')
            self.Fa = simplify(self.Q_in + self.R.T@self.Q_de)
            if print_status: print('Generating and simplifiying reduced q_dot_dot')
            self.q_dotdot_in_expr = simplify(self.H.pinv() @ (self.Fa - self.K@self.q_dot_in))
            if print_status: print('Reduced model completed')
            self.RHS_reduced = Matrix(list(self.q_dot_in)+list(self.q_dotdot_in_expr))
        else:
            if print_status: print('Generating and simplifiying reduced q_dot_dot')
            self.q_dotdot_expr = simplify(self.M.pinv() @ self.Q)
            if print_status: print('Reduced model completed')
            self.RHS = Matrix(list(self.q_dot)+list(self.q_dotdot_expr))
    def calculate_RHS(self):
        if not hasattr(self, 'RHS'):
            if self.print_status: print('Generating and simplifiying Right Hand Side')
            self.q_dotdot_de_expr = simplify(self.R_dot @ self.q_dot_in + self.R @ self.q_dotdot_in_expr)
            self.RHS = Matrix(list(self.q_dot)+list(self.q_dotdot_in_expr)+list(self.q_dotdot_de_expr))
        return self.RHS
    def calculate_lambda_vec(self):
        if not hasattr(self, 'lambda_vec'):
            if len(self.coneqs) == 0:
                self.lambda_vec = []
            else:
                if self.print_status: print('Generating and simplifiying Lambdas vector')
                self.lambda_vec = simplify(self.phi_q_de_inv@(self.Q_de- self.M_de@self.q_dotdot_de_expr))
        return self.lambda_vec
    def calculate_RHS_full(self):
        if not hasattr(self, 'RHS_full'):
            if not hasattr(self, 'RHS'): self.calculate_RHS()
            if not hasattr(self, 'lambda_vec'): self.calculate_lambda_vec()
            self.RHS_full = Matrix(list(self.RHS)+list(self.lambda_vec))
        return self.RHS_full

In [ ]:
LM_simp = SimpLagrangesMethod(lag_f, [x, y, psi, fi0, fi1, fi2, fi3] ,
                              forcelist=FL, frame=N, nonhol_coneqs=non_hol_rest)

In [ ]:
LM_simp.RHS_reduced

In [ ]:
LM_simp.calculate_RHS_full()

In [ ]:
LM_simp_no_const = SimpLagrangesMethod(lag_f, [x, y, psi, fi0, fi1, fi2, fi3] ,
                              forcelist=FL, frame=N, nonhol_coneqs=None)

In [ ]:
LM_simp_no_const.calculate_RHS_full()

In [ ]:
res = LM.rhs()

In [ ]:
big_mat = LM.mass_matrix_full[7:, 7:]
big_mat

In [ ]:
big_mat_num = big_mat.subs([
    [m, 15.75],
    [l0, 0.2096],
    [l1, 0.2096],
    [i_r, 0.461],
    [i_w, 0.00266],
    [r, 0.0667]
])
big_mat_num

In [ ]:
big_mat_num.pinv()